

---


Nombre de los integrantes


*   Cruz Pérez Joshua Santiago
*   Hernández Banda Oziel
*   Jimenez Borzani Naomi Daniela
*   Paredes Hernández Ximena


---




24. Sea el sistema $A \bar{x}=\bar{b}$ dado por $A$ igual a

$$
\left[\begin{array}{ccccccc}
9 & -4 & 1 & 0 & \cdots & \cdots & 0 \\
-4 & 6 & -4 & 1 & \ddots & & \vdots \\
1 & -4 & 6 & -4 & 1 & \ddots & \vdots \\
0 & \ddots & \ddots & \ddots & \ddots & \ddots & 0 \\
\vdots & \ddots & 1 & -4 & 6 & -4 & 1 \\
\vdots & & \ddots & 1 & -4 & 5 & -2 \\
0 & \cdots & \cdots & 0 & 1 & -2 & 1
\end{array}\right]
$$

tomando $\bar{b}=[1,1, \cdots, 1]^T$.

Para $n=100$ resolver por:


(a) Factorización $L U$.

In [ ]:
import numpy as np

# ============================
# SUSTITUCIÓN HACIA DELANTE
# ============================
def SustitucionDelante(Mat, b):
    """
    Realiza la sustitución hacia adelante para resolver Ly = b,
    donde L es una matriz triangular inferior.

    Parámetros:
    - Mat: matriz triangular inferior L
    - b: vector del lado derecho

    Retorna:
    - x: solución del sistema
    """
    n = Mat.shape[0]
    x = np.zeros(n)
    for i in range(n):
        SumCum = 0.0
        for j in range(i):
            SumCum += Mat[i, j] * x[j]
        x[i] = (b[i] - SumCum) / Mat[i, i]
    return x

# ============================
# SUSTITUCIÓN HACIA ATRÁS
# ============================
def SustitucionAtras(Mat, b):
    """
    Realiza la sustitución hacia atrás para resolver Ux = y,
    donde U es una matriz triangular superior.

    Parámetros:
    - Mat: matriz triangular superior U
    - b: vector del lado derecho (resultado de Ly)

    Retorna:
    - x: solución del sistema
    """
    n = Mat.shape[0]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        SumCum = 0.0
        for j in range(i+1, n):
            SumCum += Mat[i, j] * x[j]
        x[i] = (b[i] - SumCum) / Mat[i, i]
    return x

# ============================
# DESCOMPOSICIÓN LU
# ============================
def LU(A):
    """
    Realiza la descomposición LU sin pivoteo parcial.
    A = LU, donde L es inferior y U es superior.

    Parámetros:
    - A: matriz cuadrada a descomponer

    Retorna:
    - L: matriz triangular inferior
    - U: matriz triangular superior
    """
    U = np.copy(A)
    L = np.eye(A.shape[0])  # Matriz identidad del mismo tamaño
    for i in range(A.shape[0]):
        Li = np.eye(A.shape[0])  # Matriz elemental
        for j in range(i+1, A.shape[0]):
            Li[j, i] = -U[j, i] / U[i, i]
            L[j, i] = U[j, i] / U[i, i]
        U = Li @ U  # Producto matricial
    return L, U

# ============================
# SOLUCIÓN DE SISTEMAS CON LU
# ============================
def SolverLU(A, b):
    """
    Resuelve el sistema Ax = b usando la descomposición LU.

    Parámetros:
    - A: matriz de coeficientes
    - b: vector del lado derecho

    Retorna:
    - x: solución del sistema
    """
    L, U = LU(A)
    y = SustitucionDelante(L, b)  # Resolver Ly = b
    x = SustitucionAtras(U, y)    # Resolver Ux = y
    return x

# ============================
# CONSTRUCCIÓN DE MATRIZ A
# ============================
def construir_matriz_A(n):
    """
    Construye una matriz tridiagonal extendida con condiciones específicas
    para un sistema de tamaño n.

    Parámetros:
    - n: tamaño de la matriz (n x n)

    Retorna:
    - A: matriz construida
    """
    A = np.zeros((n, n))
    for i in range(n):
        if i >= 2:
            A[i, i-2] = 1       # Segundo subdiagonal
        if i >= 1:
            A[i, i-1] = -4      # Primer subdiagonal
        A[i, i] = 6 if 0 < i < n-1 else 9 if i == 0 else 1  # Diagonal principal
        if i + 1 < n:
            A[i, i+1] = -4      # Primer superdiagonal
        if i + 2 < n:
            A[i, i+2] = 1       # Segundo superdiagonal

    # Ajustes específicos a las últimas filas
    A[-2, -3] = -4
    A[-2, -2] = 5
    A[-2, -1] = -2
    A[-1, -2] = -2
    A[-1, -1] = 1
    return A

# ============================
# EJECUCIÓN DEL PROGRAMA
# ============================

n = 100                           # Tamaño del sistema
A = construir_matriz_A(n)         # Construcción de la matriz A
b = np.ones(n)                    # Vector del lado derecho b = [1, 1, ..., 1]

Sol = SolverLU(A, b)              # Resolución del sistema Ax = b

# Mostrar matriz y solución
print("Nuestra matriz:\n", construir_matriz_A(100))
print("Solución del sistema:\n", Sol)


(b) Crear una rutina (tipo banda) para la estructura de la matriz y resolver. Comparar y
comentar 24a y 24b.

In [ ]:
import numpy as np

# ========================================
# CONFIGURACIÓN INICIAL
# ========================================

n = 100  # Tamaño de la matriz (n x n)

# Inicialización de las cinco diagonales de la matriz banda
DP = np.zeros(n)     # Diagonal principal
DI = np.zeros(n-1)   # Primera diagonal inferior
DS = np.zeros(n-1)   # Primera diagonal superior
EI = np.zeros(n-2)   # Segunda diagonal inferior
ES = np.zeros(n-2)   # Segunda diagonal superior

# ========================================
# LLENADO DE DIAGONALES SEGÚN CONDICIONES
# ========================================
for i in range(n):
    if i == 0:
        DP[i] = 9
        DS[i] = -4
        ES[i] = 1
    elif i == 1:
        DI[i-1] = -4
        DP[i] = 6
        DS[i] = -4
        ES[i] = 1
    elif i == n - 2:
        EI[i-2] = 1
        DI[i-1] = -4
        DP[i] = 5
        DS[i] = -2
    elif i == n - 1:
        EI[i-2] = -2
        DI[i-1] = -2
        DP[i] = 1
    else:
        EI[i-2] = 1 if i >= 2 else EI[i-2]
        DI[i-1] = -4
        DP[i] = 6
        DS[i] = -4
        ES[i] = 1 if i < n - 2 else ES[i]

# Vector del lado derecho del sistema
b = np.ones(n)

# ========================================
# ALGORITMO DE THOMAS MODIFICADO (PENTADIAGONAL)
# ========================================
def Thomas_Modificado(DP, DS, DI, ES, EI, b):
    """
    Algoritmo de Thomas modificado para resolver sistemas lineales
    con matrices pentadiagonales (bandas de 5).

    Parámetros:
    - DP: Diagonal principal
    - DS: Primera diagonal superior
    - DI: Primera diagonal inferior
    - ES: Segunda diagonal superior
    - EI: Segunda diagonal inferior
    - b: Vector del lado derecho

    Retorna:
    - x: solución del sistema Ax = b
    """
    n = len(DP)

    # Copias para evitar modificar los vectores originales
    a = EI.copy()
    b1 = DI.copy()
    c = DP.copy()
    d = DS.copy()
    e = ES.copy()
    f = b.copy()

    # Vectores auxiliares
    alpha = np.zeros(n)  # Segunda superdiagonal modificada
    beta = np.zeros(n)   # Primera superdiagonal modificada
    gamma = np.zeros(n)  # Diagonal principal modificada
    x = np.zeros(n)      # Solución final

    # ============================
    # FASE DE ELIMINACIÓN HACIA ADELANTE
    # ============================
    # Paso base para los primeros dos elementos
    gamma[0] = c[0]
    beta[0] = d[0] / gamma[0]
    alpha[0] = e[0] / gamma[0]
    f[0] = f[0] / gamma[0]

    gamma[1] = c[1] - b1[0] * beta[0]
    beta[1] = (d[1] - b1[0] * alpha[0]) / gamma[1]
    alpha[1] = e[1] / gamma[1]
    f[1] = (f[1] - b1[0] * f[0]) / gamma[1]

    # Eliminación hacia adelante para el resto
    for i in range(2, n):
        gamma[i] = c[i] - b1[i-1] * beta[i-1] - a[i-2] * alpha[i-2]
        if i < n-1:
            beta[i] = (d[i] - b1[i-1] * alpha[i-1]) / gamma[i]
        if i < n-2:
            alpha[i] = e[i] / gamma[i]
        f[i] = (f[i] - b1[i-1] * f[i-1] - a[i-2] * f[i-2]) / gamma[i]

    # ============================
    # SUSTITUCIÓN HACIA ATRÁS
    # ============================
    x[n-1] = f[n-1]
    x[n-2] = f[n-2] - beta[n-2] * x[n-1]

    for i in range(n-3, -1, -1):
        x[i] = f[i] - beta[i] * x[i+1] - alpha[i] * x[i+2]

    return x

# ========================================
# RESOLVER SISTEMA Y MOSTRAR RESULTADO
# ========================================
sol = Thomas_Modificado(DP, DS, DI, ES, EI, b)

print("Solución del sistema Ax = b usando Thomas modificado (tipo banda):")
print(sol)


(c) Verificar que la matriz $A$ tiene una factización de la forma $A=R R^T$ donde $R$ es una matriz triangular superior de la forma

$$
\left[\begin{array}{cccccc}
2 & -2 & 1 & 0 & \cdots & 0 \\
0 & 1 & -2 & 1 & \ddots & \vdots \\
\vdots & \ddots & \ddots & \ddots & \ddots & 0 \\
\vdots & & \ddots & 1 & -2 & 1 \\
\vdots & & & \ddots & 1 & -2 \\
0 & \cdots & \cdots & \cdots & 0 & 1
\end{array}\right]
$$

Crear una rutina con la factorización $R R^T$. Para $n=1000$ resolver el sistema. Comparar y resolver $24 \mathrm{a}, 24 \mathrm{~b}$ y 24 c .

In [ ]:
import numpy as np
from numpy import linalg as LA

# ========================================
# SUSTITUCIÓN HACIA ADELANTE
# ========================================
def SustDelante(L, b):
    """
    Realiza sustitución hacia adelante para resolver Ly = b,
    donde L es una matriz triangular inferior.

    Parámetros:
    - L: matriz triangular inferior
    - b: vector del lado derecho

    Retorna:
    - x: solución del sistema
    """
    x = np.zeros_like(b)
    n = L.shape[0]
    for i in range(n):
        suma = 0.0
        for j in range(i):
            suma += L[i, j] * x[j]
        x[i] = (b[i] - suma) / L[i, i]
    return x

# ========================================
# SUSTITUCIÓN HACIA ATRÁS
# ========================================
def SustAtras(U, y):
    """
    Realiza sustitución hacia atrás para resolver Ux = y,
    donde U es una matriz triangular superior (usualmente Lᵗ).

    Parámetros:
    - U: matriz triangular superior
    - y: vector resultado de Ly = b

    Retorna:
    - x: solución del sistema
    """
    x = np.zeros_like(y)
    n = U.shape[0]
    x[n-1] = y[n-1] / U[n-1, n-1]
    for i in range(n-2, -1, -1):
        suma = 0.0
        for j in range(i+1, n):
            suma += U[i, j] * x[j]
        x[i] = (y[i] - suma) / U[i, i]
    return x

# ========================================
# FACTORIZACIÓN DE CHOLESKY
# ========================================
def Cholesky(A):
    """
    Realiza la factorización de Cholesky para una matriz simétrica definida positiva.
    A = LLᵗ, donde L es triangular inferior.

    Parámetros:
    - A: matriz simétrica definida positiva

    Retorna:
    - L: matriz triangular inferior
    """
    n = A.shape[0]
    L = np.zeros_like(A)
    for i in range(n):
        for j in range(i+1):
            if i == j:
                suma = sum(L[i][k]**2 for k in range(i))
                L[i][i] = np.sqrt(A[i][i] - suma)
            else:
                suma = sum(L[i][k] * L[j][k] for k in range(j))
                L[i][j] = (A[i][j] - suma) / L[j][j]
    return L

# ========================================
# RESOLVER SISTEMA Ax = b USANDO CHOLESKY
# ========================================
def SolveChol(A, b):
    """
    Resuelve el sistema Ax = b utilizando factorización de Cholesky.

    Parámetros:
    - A: matriz simétrica definida positiva
    - b: vector del lado derecho

    Retorna:
    - x: solución del sistema
    """
    L = Cholesky(A)          # A = LLᵗ
    y = SustDelante(L, b)    # Resolver Ly = b
    x = SustAtras(L.T, y)    # Resolver Lᵗx = y
    return x

# ========================================
# CONSTRUCCIÓN DE MATRIZ R
# ========================================
def construir_R(n):
    """
    Construye una matriz R con estructura de banda:
    - Diagonal principal: 2 (primer elemento), luego 1
    - Primera superdiagonal: -2
    - Segunda superdiagonal: 1

    Parámetros:
    - n: tamaño de la matriz

    Retorna:
    - R: matriz (n x n)
    """
    R = np.zeros((n, n))
    for i in range(n):
        R[i, i] = 2 if i == 0 else 1
        if i + 1 < n:
            R[i, i+1] = -2
        if i + 2 < n:
            R[i, i+2] = 1
    return R

# ========================================
# PROGRAMA PRINCIPAL
# ========================================
n = 1000                     # Tamaño del sistema
R = construir_R(n)          # Construcción de matriz R
A = R @ R.T                 # A = RRᵗ, matriz simétrica definida positiva
b = np.ones(n)              # Vector b = [1, 1, ..., 1]

sol = SolveChol(A, b)       # Resolver el sistema Ax = b

# Mostrar la solución
print("Solución:\n", sol)
